TODO:

Change negative values used in minmax to 0


Import packages and set things up


In [58]:
import numpy as np
import pandas as pd
import os
import csv
import composite as ci
import inspect
from functools import reduce
import plotly.express as px

# from xml.etree import ElementTree as ET
# import requests
# import json
# import webbrowser

In [2]:
# show all output from cells
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # last_expr

In [3]:
# create output folder if not there
if not os.path.exists("../output"):
    os.makedirs("../output")

#### Country names dict


In [4]:
# load countries dictionary
with open("../data/countryAbbr.csv") as csv_file:
    reader = csv.reader(csv_file)
    country_to_abbrev = dict(reader)
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))
# add additional abbreviations
abbrev_to_country["EL"] = "Greece"
abbrev_to_country["GB"] = "United Kingdom"

In [5]:
allEurope = pd.read_csv("../data/Countries-Europe.csv")
allEurope = allEurope.name.to_list()

In [6]:
# fmt: off
countries = [
    "Belgium", "Bulgaria", "Cyprus", "Greece", "Germany", "Croatia",
    "Italy", "Denmark", "Estonia", "Spain", "Finland", "France",
    "Ireland", "Lithuania", "Latvia", "Malta", "Netherlands", "Poland",
    "Portugal", "Romania", "Sweden", "United Kingdom",]
countryAbb = [
    x if x not in country_to_abbrev else country_to_abbrev[x] for x in countries
]
# fmt: on

In [7]:
def missingCountries(df, countries=countries):
    ### check missing countries in dataset
    missing = []
    for i in countries:
        if i not in df.index.unique():
            missing.append(i)
    if len(missing) > 0:
        print("Missing countries:\n", "\n".join(missing))
    else:
        print("No missing countries")

### SDG Official Data


In [8]:
# SDG14 indicators from the UNstats
# https://unstats.un.org/sdgs/dataportal/database

sdg14 = pd.read_excel("../data/Goal14_april2023.xlsx", sheet_name=0)
# fmt: off
sdg14 = sdg14[
    [ 
        "Target", "Indicator", "SeriesCode", "GeoAreaName", 
        "TimePeriod", "Value", "Units", "SeriesDescription" 
    ]
]
# fmt: on

sdg14.Value.replace("N", np.nan, inplace=True)
sdg14.Value = sdg14.Value.astype(np.float64)

# filter countries
# sdg14 = sdg14[sdg14['GeoAreaName'].isin(countries)]
# show indicators
pd.DataFrame(
    sdg14.groupby(["Indicator", "SeriesCode", "SeriesDescription", "Units"]).size()
)

0
Indicator SeriesCode        SeriesDescription                                  Units        
14.1.1    EN_MAR_BEALITSQ   Beach litter per square kilometer (Number)         NUMBER    611
          EN_MAR_BEALIT_BP  Beach litter originating from national land-bas... PERCENT   900
          EN_MAR_BEALIT_BV  Beach litter originating from national land-bas... TONNES    900
          EN_MAR_BEALIT_EXP Exported beach litter originating from national... TONNES    900
          EN_MAR_BEALIT_OP  Beach litter originating from national land-bas... PERCENT   900
          EN_MAR_BEALIT_OV  Beach litter originating from national land-bas... TONNES    900
          EN_MAR_CHLANM     Chlorophyll-a anomaly, remote sensing (%)          PERCENT  2784
          EN_MAR_CHLDEV     Chlorophyll-a deviations, remote sensing (%)       PERCENT  4131
          EN_MAR_PLASDD     Floating plastic debris density (count per km2)    NUMBER      3
14.2.1    EN_SCP_ECSYBA     Number of countries using ecosystem-based appro... NUMBER     33
14.3.1    ER_OAW_MNACD      Average marine acidity (pH) measured at agreed ... PH        903
14.4.1    ER_H2O_FWTL       Proportion of fish stocks within biologically s... PERCENT   422
14.5.1    ER_MRN_MPA        Average proportion of Marine Key Biodiversity A... PERCENT  6049
14.6.1    ER_REG_UNFCIM     Progress by countries in the degree of implemen... NUMBER    882
14.7.1    EN_SCP_FSHGDP     Sustainable fisheries as a proportion of GDP       PERCENT  5880
14.a.1    ER_RDE_OSEX       National ocean science expenditure as a share o... PERCENT   159
14.b.1    ER_REG_SSFRAR     Degree of application of a legal/regulatory/pol... NUMBER    882
14.c.1    ER_UNCLOS_IMPLE   Score for the implementation of UNCLOS and its ... PERCENT    63
          ER_UNCLOS_RATACC  Score for the ratification of and accession to ... PERCENT    63

#### Check official data by indicator


In [9]:
sdg14check = sdg14[sdg14["GeoAreaName"].isin(countries)]
sdg14check.loc[
    sdg14check["GeoAreaName"].str.contains(r"^(?=.*United)(?=.*Kingdom)"), "GeoAreaName"
] = "United Kingdom"
sdg14check = sdg14check[sdg14check["SeriesCode"] == "EN_MAR_BEALITSQ"].pivot_table(
    columns="TimePeriod", index="GeoAreaName", values="Value", aggfunc="mean"
)
sdg14check
missingCountries(sdg14check)

TimePeriod,2015,2016,2017,2018,2019,2020
GeoAreaName,,,,,,
Belgium,1.520290e+07,2.704512e+05,2.273158e+05,1.915215e+04,6.645361e+05,NaN
Bulgaria,2.319362e+05,1.924616e+05,2.292929e+06,2.618936e+06,2.589610e+06,2.159091e+06
Croatia,4.538235e+07,4.292519e+05,NaN,NaN,5.534965e+04,NaN
Cyprus,3.804995e+05,1.230420e+06,4.879745e+06,4.901154e+06,NaN,1.879470e+07
Denmark,1.289859e+05,1.961387e+05,6.390451e+04,2.080706e+05,2.490390e+05,1.977920e+05
Estonia,NaN,NaN,1.058452e+06,NaN,NaN,NaN
Finland,NaN,NaN,2.172614e+04,1.652134e+05,NaN,NaN
France,6.693829e+06,4.550833e+06,1.906840e+05,1.066566e+06,1.329666e+06,6.046173e+05
Germany,1.010614e+05,2.106574e+06,4.334803e+04,3.861386e+05,1.695590e+04,NaN


Missing countries:
 Lithuania
Latvia
United Kingdom


### 14.1

#### (a) Index of coastal eutrophication

We use Gross Nitrogen Balance: Max-Min transformation where the max-value is the maximum value in the period 2012-2021 and the min-value is zero.


##### Gross nutrient balance (Nitrogen kg/ha), Eurostat

[Source](https://ec.europa.eu/eurostat/databrowser/view/AEI_PR_GNB__custom_153613/)


In [336]:
# Gross nutrient balance (Nitrogen kg/ha), Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/AEI_PR_GNB__custom_153613/

nitro = pd.read_csv("../data/aei_pr_gnb.csv")
nitro["geo"] = nitro["geo"].map(abbrev_to_country).fillna(nitro["geo"])
nitro = nitro[nitro["geo"].isin(countries)]
nitro = nitro.pivot_table(
    columns="TIME_PERIOD", index="geo", values="OBS_VALUE", aggfunc="mean"
)
mr = nitro.columns[-1]  # most recent year
# maxMin transformation as (max-x)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
nitro = (
    (nitro.loc[:, "2012":mr].max().max() - nitro.loc[:, "2012":mr])
    / (nitro.loc[:, "2012":mr].max().max() - nitro.loc[:, "2012":mr].min().min())
    * 100
).round(2)
nitro[[2012, 2016, mr]]

missingCountries(nitro)

TIME_PERIOD,2012,2016,2018
geo,,,
Belgium,23.68,NaN,NaN
Bulgaria,76.90,55.91,60.07
Croatia,49.10,71.89,61.95
Cyprus,5.19,NaN,NaN
Denmark,50.40,NaN,NaN
...,...,...,...
Portugal,68.08,66.83,67.37
Romania,80.44,88.36,100.00
Spain,72.69,70.28,NaN


No missing countries


#### (b) Floating Plastic Debris Density

We use two indicators:

1. Plastic Waste kg/ha: Max-Min Transformation where the max-value and min-values are the maximum and minimum in the period 2012-2021.

2. Recovery Rate of Plastic Packaging: Without further transformation as score is provided dimensionless.


##### 1. Plastic Waste kg/ha, Eurostat

[Source](https://ec.europa.eu/eurostat/databrowser/view/ENV_WASGEN/)


In [345]:
# Plastic Waste kg/ha, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_WASGEN/

wasteG = pd.read_csv("../data/env_wasgen.csv")
wasteG["geo"] = wasteG["geo"].map(abbrev_to_country).fillna(wasteG["geo"])
wasteG = wasteG[wasteG["geo"].isin(countries)]

wasteG = wasteG[wasteG["unit"] == "KG_HAB"].pivot_table(
    columns="TIME_PERIOD", index="geo", values="OBS_VALUE", aggfunc="mean"
)
mr = wasteG.columns[-1]  # most recent year
# maxMin transformation as (max-x)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
wasteG = (
    (wasteG.loc[:, "2012":mr].max().max() - wasteG.loc[:, "2012":mr])
    / (wasteG.loc[:, "2012":mr].max().max() - wasteG.loc[:, "2012":mr].min().min())
    * 100
).round(2)
wasteG = wasteG.ffill(axis=1)  # fill empty values with last available year
wasteG[[2012, 2016, mr]]
missingCountries(wasteG)

TIME_PERIOD,2012,2016,2020
geo,,,
Belgium,54.62,51.26,31.93
Bulgaria,92.44,78.99,59.66
Croatia,96.64,90.76,78.15
Cyprus,100.00,98.32,88.24
Denmark,88.24,87.39,84.03
...,...,...,...
Portugal,89.92,78.99,68.07
Romania,84.03,92.44,90.76
Spain,84.03,90.76,88.24


No missing countries


##### 2. Recovery rates for packaging waste, Plastic Packaging

[Source](https://ec.europa.eu/eurostat/databrowser/view/ten00062/)


In [347]:
# Recovery rates for packaging waste, Plastic Packaging, Percentage, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ten00062/

wasteR = pd.read_csv("../data/ten00062.csv")
wasteR["geo"] = wasteR["geo"].map(abbrev_to_country).fillna(wasteR["geo"])
wasteR = wasteR[wasteR["geo"].isin(countries)]
wasteR = wasteR.pivot_table(
    columns="TIME_PERIOD", index="geo", values="OBS_VALUE", aggfunc="mean"
)
mr = wasteR.columns[-1]  # most recent year
wasteR = wasteR.ffill(axis=1)  # fill empty values with last available year
wasteR[[2012, 2016, mr]]
missingCountries(wasteR)

TIME_PERIOD,2012,2016,2020
geo,,,
Belgium,92.7,99.5,99.4
Bulgaria,44.1,52.6,50.6
Croatia,45.4,41.1,34.1
Cyprus,44.8,74.3,51.9
Denmark,99.4,98.1,73.1
...,...,...,...
Portugal,39.4,49.9,56.9
Romania,51.9,49.9,37.0
Spain,53.2,61.8,55.5


No missing countries


### 14.2

#### Proportion of national exclusive economic zones managed using ecosystem-based approaches

As of now, it is a binary variable: Number of countries using ecosystem-based approaches to manage marine areas

Data is scarce and the next round of data collection is in 2025. More information [here](https://unstats.un.org/sdgs/metadata/files/Metadata-14-02-01.pdf)

Spatial comparison within the EU is not expected to yield great differences and we neglect target 14.2.


### 14.3

Average marine acidity (pH) measured at agreed suite of representative sampling stations

We use two indicators:

1. Greenhouse gas emissions under the Effort Sharing Decision (ESD): Max-Min transformation where the max- and min-values are the maximum and
   minimum in the assessment period (2012-2021)

2. Carbon Emissions per capita: Max-Min Transformation where the max-value and min-values are the maximum and minimum in the period 2012-2021.


#### 1. Greenhouse gas emissions under the Effort Sharing Decision (ESD)

Several sources (see code)


In [349]:
# Greenhouse gas emissions under the Effort Sharing Decision (ESD), Million tonnes CO2 equivalent (Mt CO2 eq), European Environment Agency
# https://www.eea.europa.eu/data-and-maps/data/esd-4

ghgESD = pd.read_excel("../data/EEA_ESD-GHG_2022.xlsx", sheet_name=1, skiprows=11)
ghgESD = ghgESD[ghgESD["Geographic entity"].isin(countries)]
ghgESD = ghgESD.set_index("Geographic entity")
ghgESD = ghgESD.dropna(axis=1, how="all")

mr = ghgESD.columns[-1]  # most recent year
ghgESD = ghgESD * 1_000_000
ghgESD

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Geographic entity,,,,,,,,,,,,,,,,,
Belgium,7.894226e+07,7.769173e+07,7.676470e+07,7.920489e+07,7.582344e+07,7.888709e+07,7.351803e+07,7.394155e+07,74264633.0,70054910.0,7.271952e+07,7.406315e+07,7.082456e+07,7.425386e+07,7.201355e+07,64904157.0,6.952621e+07
Bulgaria,2.415308e+07,2.484923e+07,2.564486e+07,2.531097e+07,2.305398e+07,2.425483e+07,2.313565e+07,2.323988e+07,22238074.0,22900867.0,2.535487e+07,2.558795e+07,2.652679e+07,2.633923e+07,2.581452e+07,25735613.0,2.548095e+07
Croatia,1.719954e+07,1.753447e+07,1.786322e+07,1.787936e+07,1.713561e+07,1.737437e+07,1.719106e+07,1.628062e+07,15125525.0,14663196.0,1.556530e+07,1.600681e+07,1.666930e+07,1.621917e+07,1.605824e+07,16518244.0,1.623321e+07
Cyprus,4.137555e+06,4.220011e+06,4.416551e+06,4.457502e+06,4.435488e+06,4.415135e+06,4.562467e+06,4.250998e+06,3938120.0,3924856.0,4.060621e+06,4.111441e+06,4.270890e+06,4.162760e+06,4.377563e+06,4243155.0,4.604652e+06
Denmark,4.039735e+07,4.037132e+07,4.047414e+07,3.975169e+07,3.804919e+07,3.836427e+07,3.695189e+07,3.567756e+07,33705936.0,32643514.0,3.252022e+07,3.312468e+07,3.267691e+07,3.314244e+07,3.205059e+07,30835287.0,2.937253e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Portugal,4.700460e+07,4.556860e+07,4.516036e+07,4.488679e+07,4.359351e+07,4.347835e+07,4.139132e+07,3.923774e+07,38610318.0,38836638.0,4.061406e+07,4.157259e+07,4.018637e+07,4.057186e+07,4.152706e+07,38536930.0,4.048186e+07
Romania,7.491422e+07,7.630262e+07,7.306953e+07,7.665722e+07,7.099121e+07,6.802843e+07,7.109107e+07,7.279826e+07,72718616.0,72534134.0,7.455538e+07,7.312304e+07,7.536325e+07,7.765366e+07,7.521134e+07,77123535.0,8.227412e+07
Spain,2.381348e+08,2.418053e+08,2.462463e+08,2.355816e+08,2.229995e+08,2.245418e+08,2.135237e+08,2.041969e+08,200277677.0,199755020.0,1.961532e+08,1.984722e+08,2.011074e+08,2.030298e+08,2.018787e+08,184188341.0,1.948029e+08


In [354]:
# Build ESD allocation with two sources and interpolation for years previous to 2013

# Allocation for 2020 target
# https://ec.europa.eu/clima/ets/esdAllocations.do?languageCode=en&esdRegistry=-1&esdYear=&search=Search&currentSortSettings=
allocation2020 = pd.read_xml(
    "../data/esdAllocations2020.xml", xpath=".//ESDAllocations/ESDAllocationInfo"
)
allocation2020["Country"] = (
    allocation2020["ESDMemberState"]
    .map(abbrev_to_country)
    .fillna(allocation2020["ESDMemberState"])
)
allocation2020 = allocation2020[allocation2020["Country"].isin(countries)]
allocation2020 = allocation2020.pivot_table(
    columns="ESDYear", index="Country", values="Allocation", aggfunc="mean"
)

# Allocation for 2030 target
# https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32020D2126
allocation2030 = pd.read_html("../data/esdAllocations2030.html")[13][1:]
allocation2030.columns = allocation2030.iloc[0]
allocation2030 = allocation2030[1:]
allocation2030.loc[
    allocation2030["Member State"].str.contains("Netherlands"), "Member State"
] = "Netherlands"
allocation2030 = allocation2030[allocation2030["Member State"].isin(countries)]
allocation2030.set_index("Member State", inplace=True)
for col in allocation2030.columns:
    allocation2030[col] = allocation2030[col].apply(
        lambda x: str(x).replace("\xa0", "")
    )
allocation2030 = allocation2030.astype(int)

# Merge 2005 values with 2020 and 2030 allocations. Interpolate for years 2006-2012
allocationESD = ghgESD[[2005]].merge(
    allocation2020.merge(
        allocation2030, left_index=True, right_index=True, how="outer"
    ),
    left_index=True,
    right_index=True,
    how="outer",
)
allocationESD.columns = allocationESD.columns.astype(int)
allocationESD[list(range(2006, 2013))] = np.nan  # create empty columns for 2006-2012
allocationESD = allocationESD[list(range(2005, 2031))]
allocationESD.interpolate(axis=1, inplace=True)

# Calculate score for ESD
scoreESD = 100 - (100 * (ghgESD - allocationESD) / allocationESD)
scoreESD[scoreESD > 100] = 100
scoreESD = scoreESD.ffill(axis=1)  # fill empty values with last available year
scoreESD[[2012, 2016, 2021]]
missingCountries(scoreESD)

,2012,2016,2021
Geographic entity,,,
Belgium,100.0,99.633951,100.00000
Bulgaria,100.0,100.000000,100.00000
Croatia,100.0,100.000000,100.00000
Cyprus,100.0,100.000000,86.94582
Denmark,100.0,100.000000,100.00000
...,...,...,...
Portugal,100.0,100.000000,100.00000
Romania,100.0,100.000000,100.00000
Spain,100.0,100.000000,100.00000


No missing countries


Alternative (dif-ref) score for ESD (measuring this way does not make a lot of sense to me)


In [15]:
# Alternative (dif-ref) score for ESD (measuring this way does not make a lot of sense to me)

scoreESD2020 = 100 - 100 * (
    ghgESD.loc[:, :2020].subtract(allocationESD[2020], axis=0)
).divide(allocationESD[2020], axis=0)
scoreESD2030 = 100 - 100 * (
    ghgESD.loc[:, 2021:].subtract(allocationESD[2030], axis=0)
).divide(allocationESD[2030], axis=0)
scoreESD1 = scoreESD2020.merge(scoreESD2030, left_index=True, right_index=True)
scoreESD1[scoreESD1 > 100] = 100
scoreESD1[[2012, 2018, 2021]]

,2012,2018,2021
Geographic entity,,,
Belgium,91.656932,91.199322,68.926378
Bulgaria,100.000000,100.000000,85.870684
Croatia,100.000000,100.000000,100.000000
Cyprus,93.063293,95.282985,58.003181
Denmark,88.726856,96.633523,80.718421
Estonia,92.497465,98.385997,86.192175
Finland,87.856553,95.061850,70.558746
France,93.315120,100.000000,70.431556
Germany,93.219499,94.368820,65.038617


#### 2. Carbon emissions per capita

Several sources (see code)


In [16]:
# Population on 1 January, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/tps00001/

pop = pd.read_csv("../data/tps00001.csv")
pop["geo"] = pop["geo"].map(abbrev_to_country).fillna(pop["geo"])
pop = pop[pop["geo"].isin(countries)]
pop = pop.pivot_table(
    columns="TIME_PERIOD", index="geo", values="OBS_VALUE", aggfunc="mean"
)

In [356]:
# CO2, TOTX4_MEMONIA, THS_T thousand tonnes, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_AIR_GGE/

co2 = pd.read_csv("../data/env_air_gge.csv")
co2["geo"] = co2["geo"].map(abbrev_to_country).fillna(co2["geo"])
co2 = co2[co2["geo"].isin(countries)]
co2 = co2.pivot_table(
    columns="TIME_PERIOD", index="geo", values="OBS_VALUE", aggfunc="mean"
)

mr = co2.columns[-1]  # most recent year
co2pc = co2 / pop * 1000 * 1000  ## tonnes co2 per capita
co2pc
# maxMin transformation as (max-x)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
co2pc = (
    (co2pc.loc[:, 2012:mr].max().max() - co2pc.loc[:, 2012:mr])
    / (co2pc.loc[:, 2012:mr].max().max() - co2pc.loc[:, 2012:mr].min().min())
    * 100
).round(2)

# co2pc[[2012, 2016, mr]]

missingCountries(co2pc)

TIME_PERIOD,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
geo,,,,,,,,,,,,,,,,,,,,,
Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9564.893939,9027.680389,9384.259029,9192.649143,9155.052795,9213.232445,9131.633407,8155.779505,NaN,NaN
Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5924.200967,6317.365513,6774.230589,6447.039776,6798.518529,6295.373895,6140.487217,5377.837417,NaN,NaN
Croatia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4410.627056,4250.172306,4302.348037,4410.935342,4620.141047,4453.496815,4529.488652,4197.549878,NaN,NaN
Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8505.112730,9012.972028,9125.698931,9735.017134,9979.983669,9703.807756,9563.876657,8561.990079,NaN,NaN
Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7959.393342,7210.624756,6741.971990,7034.621397,6605.525461,6579.832180,5911.428036,5065.500691,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Portugal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4878.090992,4901.215569,5357.080825,5218.463196,5745.892677,5413.938764,5073.866234,4225.049969,NaN,NaN
Romania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3967.250071,3964.055606,3961.248469,3941.246679,4125.881716,4143.011171,3991.226023,3842.943896,NaN,NaN
Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5704.109687,5793.594923,6172.271150,5973.530332,6285.787679,6180.986050,5769.633420,4642.960982,NaN,NaN


No missing countries


In [18]:
# Data to corrobarate the ENV_AIR_GGE data. Very close numbers and we have for 2021
# Fossil CO2 emissions by country (territorial), million tonnes of carbon per year (1MtC = 1 million tonne of carbon = 3.664 million tonnes of CO2)
# https://globalcarbonbudget.org/carbonbudget/

co2T = pd.read_excel(
    "../data/National_Fossil_Carbon_Emissions_2022v1.0.xlsx", sheet_name=1, skiprows=11
)
co2T = co2T.T
co2T.columns = co2T.iloc[0, :]
co2T.columns = co2T.columns.astype(int)
co2T = co2T.rename_axis(index="geo", columns=None)
co2T = co2T[co2T.index.isin(countries)]

mr = co2T.columns[-1]  # most recent year
co2T = co2T * 3.664  # convert from carbon to co2
co2pc = co2T / pop * 1000_000  ## tonnes co2 per capita

# maxMin transformation as (max-x)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
co2pc = (
    (co2pc.loc[:, 2012:mr].max().max() - co2pc.loc[:, 2012:mr])
    / (co2pc.loc[:, 2012:mr].max().max() - co2pc.loc[:, 2012:mr].min().min())
    * 100
).round(2)
co2pc = co2pc.ffill(axis=1)  # fill empty values with last available year
co2pc[[2012, 2018, mr]]

missingCountries(co2pc)

,2012,2018,2021
geo,,,
Belgium,47.69,51.71,55.72
Bulgaria,69.89,73.34,73.62
Croatia,87.80,89.04,88.46
Cyprus,54.53,53.95,54.06
Denmark,65.29,74.85,82.76
Estonia,12.53,11.12,59.32
Finland,45.77,55.54,68.23
France,79.37,84.88,87.31
Germany,40.23,48.77,57.15


No missing countries


### 14.4

Proportion of fish stocks within biologically sustainable levels

We use two indicators:

1. FMSY/F: catch-weighted average

2. B/BMSY: catch-weighted average


#### 1. FMSY/F


#### 2. B/BMSY


### 14.5

Coverage of protected areas in relation to marine area

We consider two indicators:

1. Coverage of protected areas in relation to marine areas: Distance to Reference transformation where the max-value is set to 30% and the min-value is 0%
2. OHI 'Biodiversity' Index : No further transformation


#### 1. Marine protected areas (% of territorial waters)

[Source](https://data.worldbank.org/indicator/ER.MRN.PTMR.ZS)


In [359]:
# Marine protected areas (% of territorial waters), World Bank aggregation of https://www.protectedplanet.net/en/thematic-areas/marine-protected-areas
# https://data.worldbank.org/indicator/ER.MRN.PTMR.ZS

mpa = pd.read_csv("../data/API_ER.MRN.PTMR.ZS_DS2.csv", skiprows=4)
mpa = mpa[mpa["Country Name"].isin(countries)].set_index("Country Name")
mpa = mpa.dropna(axis=1, how="all")
mpa = mpa.drop(["Country Code", "Indicator Name", "Indicator Code"], axis=1)
# display all rows
pd.set_option("display.max_rows", None)
mpa
mpa = (mpa / 0.3).round(2)  # dis-ref with target 30%
mpa[mpa > 100] = 100
mpa.sort_index(inplace=True)
mpa[["2016", "2021"]]
missingCountries(mpa)

,2016,2017,2018,2019,2020,2021
Country Name,,,,,,
Belgium,36.651179,36.655243,36.655243,36.653420,36.653420,36.653416
Bulgaria,8.103060,8.103310,8.103310,8.108620,8.108338,8.108338
Cyprus,0.123118,0.122716,0.122716,0.123078,8.620539,8.620539
Germany,45.356552,45.357396,45.357396,45.380450,45.388630,45.460426
Denmark,17.848149,17.848993,17.848993,17.919840,18.322210,18.322140
Spain,8.729889,8.374113,8.374113,12.688430,12.758700,12.757444
Estonia,18.626329,18.620442,18.620442,18.634920,18.778840,18.778709
Finland,11.130294,10.509963,10.509963,12.110620,11.986870,11.987005
France,26.166849,45.045795,45.045795,48.928770,50.356530,49.826946


,2016,2021
Country Name,,
Belgium,100.00,100.00
Bulgaria,27.01,27.03
Croatia,28.47,29.96
Cyprus,0.41,28.74
Denmark,59.49,61.07
Estonia,62.09,62.60
Finland,37.10,39.96
France,87.22,100.00
Germany,100.00,100.00


No missing countries


#### 2. OHI 'Biodiversity' Index

[Source](https://oceanhealthindex.org/global-scores/data-download/)


In [20]:
# OHI 'Biodiversity' Index
# https://oceanhealthindex.org/global-scores/data-download/

ohiBio = pd.read_csv("../data/scoresOHI.csv")
ohiBio = ohiBio[ohiBio["region_name"].isin(countries)]
ohiBio = ohiBio[(ohiBio.long_goal == "Biodiversity") & (ohiBio.dimension == "score")]
ohiBio = ohiBio.pivot_table(
    columns="scenario", index="region_name", values="value", aggfunc="mean"
)

ohiBio[[2012, 2018, 2022]]
missingCountries(ohiBio)

scenario,2012,2018,2022
region_name,,,
Belgium,74.65,71.16,71.53
Bulgaria,76.12,69.96,66.19
Croatia,60.77,55.80,55.65
Cyprus,68.48,65.77,64.07
Denmark,72.56,69.75,71.17
Estonia,80.68,73.42,74.96
Finland,79.25,71.68,73.02
France,72.20,74.81,75.08
Germany,73.45,69.04,72.25


No missing countries


### 14.6

Degree of implementation of international instruments aiming to combat illegal, unreported and unregulated fishing

We use two indicators:

1. Fishing Subsidies relative to Landings: Max-Min Transformation where the max-value and min-value are the maximum and minimum in the assessment period (2012-2021)
2. TAC/Catch:


In [21]:
# load oecd abbreviation list
with open("../data/oecdAbb.csv") as csv_file:
    reader = csv.reader(csv_file)
    oecdAbb = dict(reader)

#### 1. Fishing Subsidies relative to Landings

Several sources


In [22]:
# Fisheries Support Estimate
# https://stats.oecd.org/Index.aspx?DataSetCode=FISH_FSE

fse = pd.read_csv("../data/FISH_FSE.csv")
fse = fse[fse["Country"].isin(countries)]

# strip variable codes and delete parent variables to avoid double counting
# solution from https://stackoverflow.com/q/76183612/14534411

separator = "."
fse["vCode"] = fse.Variable.str.rsplit(".", n=1).str.get(0)

variableAll = fse.vCode.unique().tolist()


def is_parent(p, target):
    return p.startswith(target) and len(p) > len(target) and p[len(target)] == "."


vSupport = []
prev = ""
for s in sorted(variableAll) + [""]:
    if prev and not is_parent(s, prev):
        vSupport.append(prev)
    prev = s

fse = fse[(fse.vCode.isin(vSupport)) | (fse.vCode.isna())]

# we use US dollars because some countries use their own currency ('Danish Krone', 'Zloty', 'Swedish Krona')
fse = fse[fse.Measure == "US dollar"][["Country", "Variable", "Value", "Year", "Unit"]]

# we include I.B, I.C, I.D, I.E  II. SUPPORT FOR SERVICES TO THE SECTOR (except the ones specified below)
fse = fse[
    fse.Variable.str.startswith(("I.E.1", "II."))
    & ~fse.Variable.str.startswith(("II.E", "II.F", "II.G"))
]

fse = fse.groupby(["Country", "Year"]).sum(numeric_only=True).reset_index()

# fse = fse.pivot_table(columns='Year', index='Country', values='Value', aggfunc='mean')
# fse[[2012, 2016, 2020]]

In [23]:
# Landings in USD
# https://data.oecd.org/fish/fish-landings.htm

landing = pd.read_csv("../data/fishLandingsOECD.csv")
landing["LOCATION"] = landing["LOCATION"].map(oecdAbb).fillna(landing["LOCATION"])
landing = landing[
    landing["LOCATION"].isin(countries)
    & (landing["MEASURE"] == "USD")
    & (landing["SUBJECT"] == "TOT")
]
# landing = landing.pivot_table(columns='TIME', index='LOCATION', values='Value', aggfunc='mean')
# landing = landing[list(range(2010, 2021))]
landing.rename(
    columns={"LOCATION": "Country", "TIME": "Year", "Value": "Landing"}, inplace=True
)

In [369]:
# merge subsidies-landings and calculate score
fseLanding = pd.merge(
    fse, landing, how="left", left_on=["Country", "Year"], right_on=["Country", "Year"]
)
fseLanding["fseLanding"] = fseLanding.Value / fseLanding.Landing
fseLanding = fseLanding.pivot_table(
    columns="Year", index="Country", values="fseLanding", aggfunc="mean"
)
mr = fseLanding.columns[-1]  # most recent year
# fseLanding = fseLanding[~fseLanding.index.str.contains("Poland")] Poland is an outlier
# maxMin transformation as (max-x)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
fseScore = (
    (fseLanding.loc[:, 2012:mr].max().max() - fseLanding.loc[:, 2012:mr])
    / (fseLanding.loc[:, 2012:mr].max().max() - fseLanding.loc[:, 2012:mr].min().min())
    * 100
).round(2)
fseScore = fseScore.ffill(axis=1)  # fill empty values with last available year
fseScore[[2012, 2016, mr]]


missingCountries(fseScore)

Year,2012,2016,2020
Country,,,
Belgium,99.47,99.96,99.61
Denmark,98.66,98.88,98.66
Estonia,97.82,99.78,98.83
France,99.40,99.99,99.97
Germany,NaN,100.00,99.78
Greece,99.72,100.00,99.30
Ireland,NaN,99.88,99.21
Italy,99.54,99.16,99.49
Latvia,NaN,99.31,99.86


Missing countries:
 Bulgaria
Cyprus
Croatia
Finland
Malta
Romania


In [25]:
# Alternative dataset. Difference is large
# https://www.sciencedirect.com/science/article/abs/pii/S0308597X16000026#bib4
sumaila2009 = pd.read_excel("../data/subsidies2016Sumaila.xlsx")
sumaila2009["Countries"] = sumaila2009["Countries"].str.strip()
sumaila2009 = sumaila2009[
    sumaila2009["Countries"].isin(countries) | sumaila2009["Countries"].isin(countryAbb)
]
sumaila2009.Countries = sumaila2009.Countries.map(abbrev_to_country).fillna(
    sumaila2009.Countries
)
sumaila2009 = sumaila2009[["Countries", "ReEst_Subsidy2009", "SubType"]]
sumaila2009 = sumaila2009.groupby(["Countries"]).sum(numeric_only=True)
sumaila2009

missingCountries(sumaila2009)

sumaila2018 = pd.read_csv("../data/subsidies2019Sumaila.csv")
sumaila2018 = sumaila2018[["Country", "Constant 2018 USD", "Type"]]
sumaila2018 = sumaila2018[sumaila2018.Country.isin(countries)]
sumaila2018 = sumaila2018.groupby(["Country"]).sum(numeric_only=True)
sumaila2018

missingCountries(sumaila2018)

,ReEst_Subsidy2009
Countries,
Belgium,3.000196e+04
Bulgaria,2.633312e+04
Croatia,9.133978e+02
Cyprus,1.111352e+04
Estonia,6.581073e+04
Finland,9.037797e+04
France,5.693041e+05
Germany,1.950170e+05
Greece,2.258344e+05


Missing countries:
 Denmark
Netherlands


,Constant 2018 USD
Country,
Belgium,3.499169e+07
Bulgaria,1.570597e+07
Croatia,4.364076e+07
Cyprus,2.950699e+07
Denmark,2.664440e+08
Estonia,8.575283e+07
Finland,8.503777e+07
France,3.328355e+08
Germany,1.949695e+08


No missing countries


#### 2. TAC/Catch


### 14.7

Sustainable fisheries as a proportion of GDP in small island developing States, least developed countries and all countries

We use two indicators:

1. OHI Coastal 'Livelihoods & economies' Index: No further transformation
2. OHI 'Tourism & recreation' Index: No further transformation


#### 1. OHI 'Livelihoods & economies' Index

[Source](https://oceanhealthindex.org/global-scores/data-download/)


In [26]:
# OHI 'Livelihoods & economies' Index
# https://oceanhealthindex.org/global-scores/data-download/

ohiLive = pd.read_csv("../data/scoresOHI.csv")
ohiLive = ohiLive[ohiLive["region_name"].isin(countries)]
ohiLive = ohiLive[
    (ohiLive.long_goal == "Livelihoods & economies") & (ohiLive.dimension == "score")
]
ohiLive = ohiLive.pivot_table(
    columns="scenario", index="region_name", values="value", aggfunc="mean"
)

ohiLive[[2012, 2018, 2022]]

missingCountries(ohiLive)

scenario,2012,2018,2022
region_name,,,
Belgium,67.75,80.50,80.50
Bulgaria,71.31,71.31,71.31
Croatia,78.76,78.56,78.56
Cyprus,65.03,63.18,63.18
Denmark,61.44,66.22,66.22
Estonia,59.66,73.81,73.81
Finland,75.44,82.24,82.24
France,79.40,79.41,79.41
Germany,86.50,89.31,89.31


No missing countries


#### 2. OHI 'Tourism & recreation' Index

[Source](https://oceanhealthindex.org/global-scores/data-download/)


In [27]:
# OHI 'Tourism & recreation' Index
# https://oceanhealthindex.org/global-scores/data-download/

ohiTour = pd.read_csv("../data/scoresOHI.csv")
ohiTour = ohiTour[ohiTour["region_name"].isin(countries)]
ohiTour = ohiTour[
    (ohiTour.long_goal == "Tourism & recreation") & (ohiTour.dimension == "score")
]
ohiTour = ohiTour.pivot_table(
    columns="scenario", index="region_name", values="value", aggfunc="mean"
)

ohiTour[[2012, 2018, 2022]]

missingCountries(ohiTour)

scenario,2012,2018,2022
region_name,,,
Belgium,25.64,23.87,22.89
Bulgaria,23.30,27.91,19.88
Croatia,69.36,88.66,70.06
Cyprus,60.50,58.80,59.75
Denmark,23.47,27.84,22.73
Estonia,38.02,41.89,36.82
Finland,24.10,26.95,19.94
France,44.27,47.72,45.29
Germany,71.18,72.50,61.55


No missing countries


### 14.a

Proportion of total research budget allocated to research in the field of marine technology

We use two indicators:

1. Official UNSD indicator ER_RDE_OSEX: Max-Min transformation where the max-value and min-value are the maximum and minimum in the assessment period (2009-2017) of all European countries (not restricted to the countries in the analysis).
2. SAD/TAC: Catch-weighted TAC relative to Scientific Advice

Note: ER_RDE_OSEX data comes from Global Ocean Science Report (GOSR) 2020, and goes from 2013 to 2017. Data for 2009-2012 data is available in the [UNstats archive](https://unstats.un.org/sdgs/indicators/database/archive) (download csv for 29-Mar-19)


#### 1. Ocean science expenditure ER_RDE_OSEX

Several sources


In [185]:
# EEZ area
# https://emodnet.ec.europa.eu/geonetwork/srv/eng/catalog.search#/metadata/d4a3fede-b0aa-485e-b4b2-77e8e3801fd0
eez = pd.read_csv(
    "../data/EMODnet_EEZ_v11_20210506.csv", delimiter=";", encoding="latin-1"
)
eez = eez[["Territory", "Area_km2"]].groupby(["Territory"]).sum(numeric_only=True)
eez = eez[eez.index.isin(allEurope)]

In [29]:
# %%time
# National ocean science expenditure as a share of total research and development funding (%), UNstats archive (years 2009-2012)
# https://unstats.un.org/sdgs/indicators/database/archive

# # read old data by chunks to speed loading and save 14.a.1 to separate file
# iter_csv = pd.read_csv('./data/AllData_Onward_20190329.csv', iterator=True, chunksize=1000, encoding='cp1252')
# oResearchOld = pd.concat([chunk[chunk.Indicator == '14.a.1'] for chunk in iter_csv])
# oResearchOld.to_csv('./data/archive14a1.csv', index=False)

oResearchOld = pd.read_csv("../data/archive14a1.csv")
oResearchOld = oResearchOld.pivot(
    index="GeoAreaName", columns="TimePeriod", values="Value"
)

In [370]:
# National ocean science expenditure as a share of total research and development funding (%), UNstats
# https://unstats.un.org/sdgs/dataportal/database

# read official data and merge with archive data
oResearch = sdg14[sdg14["SeriesCode"] == "ER_RDE_OSEX"].pivot_table(
    columns="TimePeriod", index="GeoAreaName", values="Value", aggfunc="mean"
)
oResearch = oResearchOld.merge(
    oResearch, left_index=True, right_index=True, how="outer"
)
# use all countries in Europe
oResearch = oResearch[oResearch.index.isin(allEurope)].dropna(how="all", axis=1)
oResearch
# fill nan of year 2013 from new report with old report and 2016 with 2017
oResearch[2013] = oResearch["2013_y"].fillna(oResearch["2013_x"])
oResearch[2016] = oResearch[2016].fillna(oResearch[2017])
oResearch = oResearch[list(range(2013, 2018))]
# weighted by EEZ area
oResearch = oResearch.merge(eez, left_index=True, right_index=True, how="outer")
for col in oResearch.drop("Area_km2", axis=1).columns:
    oResearch[col] = (
        oResearch[col] * oResearch["Area_km2"] / (oResearch["Area_km2"].sum())
    )
# maxMin transformation as (x - min)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
oResearch = (
    (oResearch.loc[:, 2013:2017] - oResearch.loc[:, 2013:2017].min().min())
    / (
        oResearch.loc[:, 2013:2017].max().max()
        - oResearch.loc[:, 2013:2017].min().min()
    )
    * 100
).round(2)
# fill nan with mean of column
for col in oResearch.columns:
    oResearch[col] = oResearch[col].fillna(oResearch[col].mean())

oResearch = oResearch[oResearch.index.isin(countries)]
oResearch[[2013, 2017]]
# 2013,2016,2017

missingCountries(oResearch)

TimePeriod,2009,2010,2011,2012,2013_x,2013_y,2014,2015,2016,2017,2018,2019,2020,2021
GeoAreaName,,,,,,,,,,,,,,
Belgium,0.10139,0.06896,0.04898,0.05072,NaN,0.37339,0.35534,0.29857,0.28028,0.26589,0.039,0.122,0.136,0.121
Bulgaria,NaN,NaN,NaN,NaN,NaN,0.08690,0.06631,0.04895,0.05712,0.06366,NaN,NaN,NaN,NaN
Croatia,NaN,NaN,NaN,NaN,4.73095,2.92606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Finland,0.13736,0.15909,0.00017,0.20056,0.19544,0.31524,0.33874,0.30814,0.31978,0.31646,NaN,NaN,NaN,NaN
France,NaN,NaN,NaN,NaN,0.79318,NaN,NaN,NaN,NaN,0.52954,NaN,NaN,0.560,0.579
Germany,NaN,NaN,NaN,NaN,0.39863,0.32170,0.30502,0.24607,0.25098,0.24542,0.273,0.247,0.282,0.338
Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.80811,NaN,NaN,NaN,NaN
Italy,0.68649,0.74768,0.86588,1.04130,1.04468,0.36694,0.35486,0.30667,0.30592,0.19400,NaN,NaN,NaN,NaN
Netherlands,NaN,NaN,NaN,NaN,NaN,0.24938,0.23610,0.20488,0.21217,0.23503,NaN,NaN,NaN,NaN


,2013,2017
Belgium,0.020000,0.000
Bulgaria,0.100000,0.060
Croatia,7.850000,12.894
Cyprus,3.858182,12.894
Denmark,3.858182,12.894
Estonia,3.858182,12.894
Finland,1.210000,1.210
France,13.300000,8.860
Germany,0.840000,0.630
Greece,3.858182,12.894


No missing countries


#### 2. SAD/TAC


In [31]:
# If a country fishes only on fish stocks where assignment of TAC follows scientific advice, it would score 100

### 14.b

Degree of application of a legal/regulatory/policy/institutional framework which recognizes and protects access rights for small‐scale fisheries

We use two indicators:

1. OHI Artisanal Fishing Opportunities Index: No further transformation
2. Percentage of Fish Species Threatened: No further transformation


#### 1. OHI 'Artisanal opportunities' Index

[Source](https://oceanhealthindex.org/global-scores/data-download/)


In [366]:
# OHI 'Artisanal opportunities' Index
# https://oceanhealthindex.org/global-scores/data-download/

ohiArt = pd.read_csv("../data/scoresOHI.csv")
ohiArt = ohiArt[ohiArt["region_name"].isin(countries)]
ohiArt = ohiArt[
    (ohiArt.long_goal == "Artisanal opportunities") & (ohiArt.dimension == "status")
]
ohiArt = ohiArt.pivot_table(
    columns="scenario", index="region_name", values="value", aggfunc="mean"
)

ohiArt[[2012, 2018, 2022]]

missingCountries(ohiArt)

scenario,2012,2018,2022
region_name,,,
Belgium,79.51,77.65,78.66
Bulgaria,56.01,55.59,63.71
Croatia,66.35,62.56,67.33
Cyprus,77.16,71.91,76.38
Denmark,70.50,77.24,74.63
Estonia,89.39,91.18,99.24
Finland,84.67,82.73,77.45
France,77.09,76.44,78.61
Germany,73.21,76.62,73.23


No missing countries


#### 2. Percentage of Fish Species Threatened

Source TBD


In [33]:
# Percentage of Fish Species Threatened

# Waiting to get IUCN API key

# I would not use time series comparison because of this: https://www.iucnredlist.org/assessment/red-list-index

# https://fishbase.mnhn.fr/country/CountryChecklist.php?c_code=056&vhabitat=threatened&csub_code=
# Last updated 2019
# We can extract the data and calculate the percentage
# https://en.wikipedia.org/wiki/ISO_3166-1_numeric

# https://data.worldbank.org/indicator/EN.FSH.THRD.NO
# Data is only for 2018

#####################

# https://stats.oecd.org/Index.aspx?DataSetCode=WILD_LIFE#
# Here we have by country updated most recent.
# Missing countries: Bulgaria,Cyprus,Croatia,Ireland,Malta,Romania,United Kingdom

threatened = pd.read_csv("../data/WILD_LIFE_10062023112434707.csv")
threatened = threatened[
    (threatened.SPEC == "FISH_TOT") & (threatened.IUCN == "THREAT_PERCENT")
]
threatened = threatened[["Country", "Value"]].set_index("Country")
threatened = threatened[threatened.index.isin(countries)].rename(
    columns={"Value": 2021}
)
threatened[2021] = 100 - threatened[2021]
threatened

missingCountries(threatened)

,2021
Country,
Belgium,79.577
Greece,90.526
Netherlands,76.289
Poland,80.159
Portugal,66.154
Sweden,84.848
Estonia,91.262
Italy,92.668
Finland,86.364


Missing countries:
 Bulgaria
Cyprus
Croatia
Ireland
Malta
Romania
United Kingdom


### 14.c

Number of countries making progress in ratifying, accepting and implementing through legal, policy and institutional frameworks, ocean-related instruments that implement international law, as reflected in the United Nations Convention on the Law of the Sea

We use two indicators:

1. Participation in agreements of the International Marine Organization (IMO Participation Rate):
2. Measures under the Marine Strategy Framework Directive:


In [34]:
# dictionary for country codes used by the GISIS
with open("../data/gisisDict.csv") as csv_file:
    reader = csv.reader(csv_file)
    gisisDict = dict(reader)

gisisDict = {v: k for k, v in gisisDict.items()}

#### 1. Participation in agreements of the International Marine Organization

[Source](https://gisis.imo.org/Public/ST/Ratification.aspx)


In [35]:
# Participation in agreements of the International Marine Organization

# https://www.imo.org/en/About/Conventions/Pages/StatusOfConventions.aspx Excel file with current status of IMO conventions
# We get the historical data from the GISIS database: https://gisis.imo.org/Public/ST/Ratification.aspx
# You need to create account to access data.

# I tried to scrape the data but I am getting errors with Selenium and bs4.
# I downloaded the html manually

gisisCountries = {k: v for k, v in gisisDict.items() if k in countries}
listIMO = []
for v in gisisCountries.values():
    link = "https://gisis.imo.org/Public/ST/Ratification.aspx?cid=" + str(v)
    listIMO.append(link)

# for i in listIMO:
#     webbrowser.open(i)

In [374]:
imoRatedf = pd.DataFrame(columns=["Country", 2012, 2018, 2021])

# loop thru the html files in the folder and extract the data
for i in range(len(os.listdir("../data/treatiesIMO/"))):
    imoRate = pd.read_html(
        "../data/treatiesIMO/Status of Treaties » Ratification of Treaties{}.html".format(
            i
        )
    )[4]
    for country in countries:
        if country in imoRate["Treaty"][0]:
            countryIMO = country
    imoRate.columns = imoRate.iloc[1]
    imoRate = imoRate[2:]
    # new column with the year of accession and denounced
    imoRate["accession"] = (
        imoRate["Date of entry into force in country"]
        .str.extract("^([^(]+)")
        .fillna("")
    )
    imoRate["denounced"] = imoRate["Date of entry into force in country"].str.extract(
        ".*\\:(.*)\\).*"
    )
    imoRate[["accession", "denounced"]] = (
        imoRate[["accession", "denounced"]]
        .apply(pd.DatetimeIndex)
        .apply(lambda x: x.dt.year)
    )
    # count the number of treaties each country accessioned and not didn't denounced by 2012, 2018 and 2021
    for i in (2012, 2018, 2021):
        imoRate[str(i)] = np.where(
            (imoRate.accession < i)
            & ((imoRate.denounced > i) | (imoRate.denounced.isna())),
            1,
            0,
        )
    imoCount = (
        countryIMO,
        imoRate["2012"].sum(),
        imoRate["2018"].sum(),
        imoRate["2021"].sum(),
    )
    imoRatedf.loc[len(imoRatedf), imoRatedf.columns] = imoCount
# calculate total possible treaties, apply dif-ref and convert to percentage
totalIMO = len(imoRate.dropna(subset=["Date of entry into force in country"]))
imoRatedf = imoRatedf.set_index("Country").sort_index()
imoRatedf = 100 - 100 * (totalIMO - imoRatedf).divide(totalIMO)
imoRatedf = imoRatedf.astype(np.float64)
imoRatedf = imoRatedf.apply(pd.to_numeric)
imoRatedf

missingCountries(imoRatedf)

,2012,2018,2021
Country,,,
Belgium,78.431373,84.313725,90.196078
Bulgaria,76.470588,80.392157,82.352941
Croatia,72.549020,74.509804,74.509804
Cyprus,68.627451,70.588235,72.549020
Denmark,80.392157,88.235294,92.156863
Estonia,76.470588,78.431373,82.352941
Finland,76.470588,86.274510,90.196078
France,84.313725,90.196078,96.078431
Germany,80.392157,88.235294,88.235294


No missing countries


#### Measures under the Marine Strategy Framework Directive

See code for notes


In [37]:
# The barplot here: https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:52018DC0562&from=EN
# Comes from https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:52018SC0393

# The analogous report for 2020 is here https://environment.ec.europa.eu/system/files/2023-04/C_2023_2203_F1_COMMUNICATION_FROM_COMMISSION_EN_V5_P1_2532109.PDF
# But the assessment is much shorter. They refer the reader to a JRC report:
# https://publications.jrc.ec.europa.eu/repository/handle/JRC129363
# That report assesses all the descriptors, but it cannot be compared to the previous assessment.
# Moreover, the source code and data are not available.

# Overall, it is hard to make an indicator for the measures taken against pressure indicators by the MS.
# Countries report different measures and data is poor.

## Indicators aggregation


Given our ratio-scale full comparable indicators,$I_{it}$, meaningful aggregation of $N$ indicators into a composite indicator $CI_t$ is obtained according to social choice theory by applying a generalized mean:

$$CI_t(\alpha_{it},I_{it},\sigma) = \left(\sum^N_{i=1}\alpha_{it}I^{\frac{\sigma-1}{\sigma}}_{it}\right)^{\frac{\sigma}{\sigma-1}} \quad  \text{for} \quad t = 2012, 2018, 2021 \text{(or most recent)}$$

with weights $\alpha_{it} > 0$ and $0 ≤ \sigma ≤ \infty $. The parameter $\sigma$ is used to quantify the elasticity of substitution between the different indicators. High (low) values of $\sigma$ imply good (poor) substitution possibilities which means that a high score in one indicator can (cannot) compensate a low score in another indicator. Consequently, high and low values of $\sigma$ correspond to concepts of weak and strong sustainability, respectively. Depending on $\sigma$, one can obtain a full class of specific function forms for the composite indicator.

We define:

$\sigma_{Target} = 0.5$ and $\sigma_{Target} = 10$


In [38]:
# fmt: off
# missing 14.2, mortality, biomass, tacCatch, sadTac, msfd
varDf = [ nitro, wasteG, wasteR,scoreESD, co2pc, mpa, ohiBio, fseScore, 
         ohiLive, ohiTour, oResearch, ohiArt, threatened, imoRatedf
]
varNames = [ "nitro", "wasteG", "wasteR", "scoreESD", "co2pc", 
            "mpa", "ohiBio", "fseScore", "ohiLive", "ohiTour",
              "oResearch", "ohiArt", "threatened", "imoRatedf",
]
# fmt: on
dictIndicators = dict(zip(varNames, varDf))

# stack variables in each dataframe
for name, df in dictIndicators.items():
    df = df.stack().to_frame().rename(columns={0: str(name)})
    df.index.names = ["Country", "Year"]
    df.reset_index(inplace=True)
    df.Year = df.Year.astype(int)
    df.set_index(["Country", "Year"], inplace=True)
    dictIndicators[name] = df
# merge all variables into one dataframe, forward and back fill by country
indicators = pd.concat(dictIndicators.values(), axis=1, join="outer")
indicators = indicators.reset_index().sort_values(["Country", "Year"])
indicators = indicators[indicators.Year.isin(list(range(2012, 2022)))]
indicators = indicators.groupby(["Country"], group_keys=False).apply(
    lambda x: x.ffill().bfill()
)
indicators = indicators.set_index(["Country", "Year"])
indicatorsL = {
    "plastic": ["wasteG", "wasteR"],
    "14.1": ["nitro", "plastic"],
    # '14.2':[''],
    "14.3": ["scoreESD", "co2pc"],
    "14.4": ["mortality", "biomass"],
    "14.5": ["mpa", "ohiBio"],
    "14.6": ["fseScore", "tacCatch"],
    "14.7": ["ohiLive", "ohiTour"],
    "14.a": ["oResearch", "sadTac"],
    "14.b": ["ohiArt", "threatened"],
    "14.c": ["imoRatedf", "msfd"],
}

# calculate composite indicators for each target importing the function from composite.py
targets = indicators.copy()
for target, indicator in indicatorsL.items():
    try:
        alpha = 1 / len(indicatorsL[target])
        df = targets[indicator]
        sigma = 10
        targets[target] = ci.compositeDF(alpha, df, sigma)
    except KeyError:
        print("Missing indicator for", target)

targets = targets[[i for i in targets if i.startswith("14")]]
targets

Missing indicator for 14.4
Missing indicator for 14.6
Missing indicator for 14.a
Missing indicator for 14.c


14.1       14.3       14.5       14.7       14.b
Country        Year                                                       
Belgium        2012  47.872645  73.370139  87.232641  46.200186  79.053327
               2013  48.789672  73.474526  86.048292  52.202161  79.163392
               2014  37.048287  75.883383  85.798948  52.808338  79.138379
               2015  36.981616  74.454387  85.778589  52.449158  79.023306
               2016  51.278646  75.098105  85.055422  51.491526  78.778095
...                        ...        ...        ...        ...        ...
United Kingdom 2017  55.796762  87.862081  84.925435  69.610536  38.997843
               2018  55.404844  88.511220  84.489188  71.031424  38.048821
               2019  55.404844  89.605287  84.007510  70.897766  37.294233
               2020  55.404844  92.172006  85.243933  69.011947  36.863702
               2021  55.404844  92.172006  85.289779  69.378426  35.928568

[220 rows x 5 columns]

### Monte Carlo simulation

In [39]:
%%time
# calculate composite score for each target importing the function from composite.py
# monte carlo for strong sustainability and one sigma for weak sustainability

scoresStrong = ci.compositeMC(targets, years=[2012, 2016, 2021], simulations=10)
scoresWeak = pd.DataFrame(ci.compositeDF(alpha = 1 / len(targets.columns), df = targets, sigma = 10), columns=['scoreWeak'])

CPU times: user 263 ms, sys: 4.26 ms, total: 268 ms
Wall time: 257 ms


## Plots

In [381]:
InteractiveShell.ast_node_interactivity = "last_expr"  # last_expr

In [390]:
# merge all data and calculate min, max, and EEZ-weigthed average for indicators and targets

data_frames = [indicators, targets, scoresStrong, scoresWeak]
fullData = reduce(
    lambda left, right: pd.merge(
        left, right, left_index=True, right_index=True, how="inner"
    ),
    data_frames,
)

eezAverage = (
    pd.DataFrame(fullData.stack().reset_index())
    .merge(eez, left_on="Country", right_on="Territory", how="left")
    .rename(columns={0: "value", "level_2": "indicator"})
)

eezAverage["valueEEZ"] = eezAverage.value * eezAverage.Area_km2
eezAverage = (
    eezAverage.groupby(["Year", "indicator"])
    .sum(numeric_only=True)
    .drop("value", axis=1)
)
eezAverage["averageEEZ"] = eezAverage.valueEEZ / eezAverage.Area_km2

In [597]:
# plot min, max, and EEZ-weigthed average for indicators

indicatorsBox = (
    fullData[fullData.index.isin([2021], level=1)]
    .stack()
    .reset_index()
    .rename(columns={0: "value", "level_2": "indicator"})
)
indicatorsBox = indicatorsBox[indicatorsBox.indicator.isin(varNames)]
indicatorsBox.Country= indicatorsBox.Country.map(country_to_abbrev)

fig = px.box(indicatorsBox, x="indicator", y="value")
# add Countries in the max and min
# create annotation list, x is the x-axis position of the annotation
annoList = []
x = 0
for s in indicatorsBox.indicator.unique():
    # get max value for indicator
    maxVal = np.max(indicatorsBox[indicatorsBox["indicator"] == s]["value"])
    # get countries with max value, if more than 4 countries, use *
    countries = indicatorsBox.loc[(indicatorsBox['value'] == maxVal)&(indicatorsBox['indicator']== s), 'Country'].values
    if len(countries) > 4:
        countries = '*'
    countries = ',<br>'.join(countries)
    annotation = dict(
        x=x,
        # y position is the max value
        y=np.max(indicatorsBox[indicatorsBox["indicator"] == s]["value"]),
        text= countries,
        yshift=30,
        showarrow=False,
    )
    annoList.append(annotation)
    x += 1


x = 0
for s in indicatorsBox.indicator.unique():
    # get max value for indicator
    minVal = np.min(indicatorsBox[indicatorsBox["indicator"] == s]["value"])
    # get countries with min value, if more than 4 countries, use *
    countries = indicatorsBox.loc[(indicatorsBox['value'] == minVal)&(indicatorsBox['indicator']== s), 'Country'].values
    if len(countries) > 4:
        countries = '*'
    countries = ',<br>'.join(countries)
    annotation = dict(
        x=x,
        # y position is the min value
        y=np.min(indicatorsBox[indicatorsBox["indicator"] == s]["value"]),
        text= countries,
        yshift=-30,
        showarrow=False,
    )
    annoList.append(annotation)
    x += 1


fig.update_layout(annotations=annoList)

fig.show()

In [498]:
# plot min, max, and EEZ-weigthed average for targets
targetsBox = (
    fullData[fullData.index.isin([2021], level=1)]
    .stack()
    .reset_index()
    .rename(columns={0: "value", "level_2": "indicator"})
)
targetsBox = targetsBox[targetsBox.indicator.isin(list(indicatorsL.keys()))]
# targetAverage = eezAverage.loc[(2021,targets.columns), :].reset_index()

fig = px.box(targetsBox, x="indicator", y="value")
# add Countries in the max and min
# create annotation list, x is the x-axis position of the annotation
annoList = []
x = 0
for s in targetsBox.indicator.unique():
    maxVal = np.max(targetsBox[targetsBox["indicator"] == s]["value"])
    annotation = dict(
        x=x,
        # y position is the max value
        y=np.max(targetsBox[targetsBox["indicator"] == s]["value"]),
        # text is the country/ies with the max value
        text=str(
            targetsBox[
                (targetsBox["value"] == maxVal) & (targetsBox["indicator"] == s)
            ]["Country"].values[0]
        ),
        yshift=10,
        showarrow=False,
    )
    annoList.append(annotation)
    x += 1


x = 0
for s in targetsBox.indicator.unique():
    minVal = np.min(targetsBox[targetsBox["indicator"] == s]["value"])
    annotation = dict(
        x=x,
        y=np.min(targetsBox[targetsBox["indicator"] == s]["value"]),
        text=str(
            targetsBox[
                (targetsBox["value"] == minVal) & (targetsBox["indicator"] == s)
            ]["Country"].values[0]
        ),
        yshift=-10,
        showarrow=False,
    )
    annoList.append(annotation)
    x += 1

fig.update_layout(annotations=annoList)
fig.show()

In [387]:
# plot comparing strong and weak sustainability
fullData.reset_index(inplace=True)

fullData["weakRank"] = fullData.groupby("Year")["scoreWeak"].rank(ascending=False)
fullData["strongRank"] = fullData.groupby("Year")["meanScore"].rank(ascending=False)

for year in [2012, 2016, 2021]:
    fig = px.scatter(
        fullData[fullData.Year == year],
        x="weakRank",
        y="strongRank",
        text="Country",
        title=str(year),
    )
    fig.update_traces(textposition="bottom right")
    fig.update_layout(
        shapes=[
            {
                "type": "line",
                "yref": "paper",
                "xref": "paper",
                "y0": 0,
                "y1": 1,
                "x0": 0,
                "x1": 1,
            }
        ]
    )
    fig.show()

## Trash bin


In [ ]:
# Get the targets for 2020 and 2030 in percentage
# Member State greenhouse gas emission limits in 2020 and 2030 compared to 2005 greenhouse gas emissions levels
# There targets for 2020 and for 2030
# https://www.eea.europa.eu/data-and-maps/figures/national-progress-towards-greenhouse-gas
# Official journals with the data can be found at (https://climate.ec.europa.eu/eu-action/effort-sharing-member-states-emission-targets_en)

# limitESR = pd.read_excel('../data/targetsESR/FIG2-154307-CLIM058-v2-Data.xlsx', sheet_name=1, skiprows=19, header=1, skipfooter=32)
# limitESR = limitESR.rename(columns={'Unnamed: 0':'Country', '(Resulting) ESR target 2030 (AR4)':'2030ESRtarget','ESR limit for 2020':'2020ESRtarget', '2005 ESD BJ':'2005Level'})
# limitESR = limitESR[['Country', '2020ESRtarget','2030ESRtarget','2005Level']]
# limitESR.set_index('Country', inplace=True)
# limitESR = limitESR[limitESR.index.isin(countries)]

# #UK is not in the dataset, we need to add from the official journal

# for i in countries:
#     if i not in limitESR.index:
#         print(i, 'is not in the dataset')

# limitESR

In [ ]:
# These data SHALL NOT BE USED. See reason on why ENV_AIR_GGE is preferable for the calculation:
# (https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Greenhouse_gas_emission_statistics_-_emission_inventories)
# (https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Greenhouse_gas_emission_statistics_-_air_emissions_accounts&oldid=551152#Greenhouse_gas_emissions)
# CO2, KG_HAB, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_AC_AINAH_R2/


# co2 = pd.read_csv('../data/env_ac_ainah_r2.csv')
# co2 = co2[co2['geo'].isin(countryAbb)]
# co2['geo'] = co2['geo'].map(abbrev_to_country).fillna(co2['geo'])
# co2 = co2.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

# mr = co2.columns[-1] # most recent year
# # co2[[2012, 2016, mr]]/1000

In [ ]:
# From 14.6, using Eurostat data for landing values.

# Even though the USD-EUR discrepancy does not affect the ratio we calculate,
# we get today's exchange rate to convert landing values to USD and have a consistent unit
# Solution source: https://stackoverflow.com/a/17250702/14534411


# r = requests.get('http://www.ecb.int/stats/eurofxref/eurofxref-daily.xml', stream=True)
# tree = ET.parse(r.raw)
# root = tree.getroot()
# namespaces = {'ex': 'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'}
# currency = 'USD'
# match = root.find('.//ex:Cube[@currency="{}"]'.format(currency.upper()), namespaces=namespaces)
# eurTOusd = float(match.attrib['rate'])

# Landings of fishery products, TOTAL FISHERY PRODUCTS, Euro, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/FISH_LD_MAIN/

# landing = pd.read_csv('../data/fish_ld_main.csv')
# landing['Country'] = landing.geo.map(abbrev_to_country).fillna(landing.geo)
# landing = landing[landing.Country.isin(countries)]
# landing = landing[['Country', 'TIME_PERIOD', 'OBS_VALUE', 'unit']]
# landing['landingUSD'] = landing.OBS_VALUE * eurTOusd
# landing.pivot_table(columns='TIME_PERIOD', index='Country', values='landingUSD', aggfunc='mean')